In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Load training data
df = pd.read_csv('/kaggle/input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv')

# Drop Loan_ID (not useful for prediction)
df.drop('Loan_ID', axis=1, inplace=True)

# Handle missing values (cleanly, no warnings)
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].fillna(df[column].mode()[0])
    else:
        df[column] = df[column].fillna(df[column].median())

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Split into features (X) and target (y)
X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']
y = le.fit_transform(y)  # Convert 'Y'/'N' to 1/0

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Validation accuracy
preds = model.predict(X_valid)
acc = accuracy_score(y_valid, preds)
print(f"Validation Accuracy: {acc:.4f}")

# Save the trained model
joblib.dump(model, 'loan_model.pkl')
print("✅ Model saved as 'loan_model.pkl'")


Validation Accuracy: 0.7561
✅ Model saved as 'loan_model.pkl'


In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder

# Load the test dataset
test_df = pd.read_csv('/kaggle/input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv')
loan_ids = test_df['Loan_ID']  # Save for final output

# Drop Loan_ID
df = test_df.drop('Loan_ID', axis=1)

# Handle missing values (same way as training)
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].fillna(df[column].mode()[0])
    else:
        df[column] = df[column].fillna(df[column].median())

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Load trained model
model = joblib.load('loan_model.pkl')

# Predict
predictions = model.predict(df)
pred_labels = ['Y' if pred == 1 else 'N' for pred in predictions]

# Save submission file
submission = pd.DataFrame({
    'Loan_ID': loan_ids,
    'Loan_Status': pred_labels
})
submission.to_csv('submission.csv', index=False)
print("📦 Predictions saved to 'submission.csv'")


📦 Predictions saved to 'submission.csv'
